In [3]:
from experta import * 

class question(Fact):
    pass
class Answer(Fact): 
    pass

class LearningAdvisor(KnowledgeEngine):
    recommendation_found = False 
    
    def recommend_action(self, action):
        print(action + "\n")

    def combine_confidence(self, cf1, cf2):
        if not (-1 <= cf1 <= 1) or not (-1 <= cf2 <= 1):
            print(f"Warning: Confidence factors out of range [-1, 1]: cf1={cf1}, cf2={cf2}. Clamping.")
            cf1 = max(-1.0, min(1.0, cf1))
            cf2 = max(-1.0, min(1.0, cf2))
        if cf1 >= 0 and cf2 >= 0:
            return cf1 + cf2 * (1 - cf1)
        elif cf1 < 0 and cf2 < 0:
            return cf1 + cf2 * (1 + cf1)
        else:
            min_abs_cf = min(abs(cf1), abs(cf2))
            if min_abs_cf == 1.0:
                return cf1 + cf2
            return (cf1 + cf2) / (1 - min_abs_cf)
    
    def display_recommendation(self, details, confidence, level, sub_skill, skill):
        output = "\n" + "="*60 + "\n"
        output += f"✨ Recommendation for learning {skill} ({sub_skill}, {level}):\n"
        output += f"Overall Confidence: {confidence:}\n"
        output += "-"*60 + "\n"
    
        if 'ide' in details:
            ide = details['ide']
            output += "🛠️ IDE:\n"
            output += f"  Name       : {ide['name']}\n"
            output += f"  Description: {ide['description']}\n"
            output += f"  URL        : {ide['url']}\n"
            output += "-"*60 + "\n"
    
        if 'learning_resources' in details:
            output += "📚 Learning Resources:\n"
            for resource in details['learning_resources']:
                output += f"  - {resource['name']} ({resource['type']}): {resource['url']}\n"
            output += "-"*60 + "\n"
    
        if 'documentation' in details:
            output += "📄 Documentation:\n"
            for doc in details['documentation']:
                output += f"  - {doc['name']}: {doc['url']}\n"
    
        output += "="*60 
        self.recommend_action(output)


    def is_of_type(self, answer, Type, valid=None):
        if Type == "multi":
            return answer in valid
        if Type == "number":
            try:
                int(answer)
                return True
            except ValueError:
                return False
        return len(answer) > 0

    def ask_user(self, question, Type, valid=None, options=None):
        while True:
            print(f"\n{question}")
            if Type == "multi" and options:
                for i, option in enumerate(options, 1):
                    print(f"{i}. {option}")
                print("➤ Please enter the option number:")
            answer = input().strip()
            if Type == "multi" and answer.isdigit():
                try:
                    answer = options[int(answer) - 1]
                except (IndexError, ValueError):
                    answer = ""
            if self.is_of_type(answer, Type, valid):
                return answer
            print("⚠️ Invalid answer. Please try again.\n")

    @DefFacts()
    def init(self):
        yield Fact(start=True)
        yield question(
            ident="skill",
            Type="multi",
            valid=["Game Development", "Data Science", "Mobile App Development", "Web Development", "Programming"],
            text="🎓 Choose the field you want to learn:",
            options=["Game Development", "Data Science", "Mobile App Development", "Web Development", "Programming"]
        )
        yield question(
            ident="sub_skill",
            Type="multi",
            valid=["Python", "Java", "C++", "JavaScript", "C#", "Ruby",
                   "HTML", "CSS", "TypeScript", "React", "Angular", "Vue.js", "Node.js", "PHP", "Python (Django)", "Laravel", "ASP.NET", "Full-stack",
                   "Android (Java)", "Flutter", "React Native", "iOS (Swift)",
                   "Unity", "Unreal Engine", 
                   "Data Analysis", "Machine Learning", "Deep Learning", "Data Visualization"],
            text="📑 Select your specialization:",
            options=["Python", "Java", "C++", "JavaScript", "C#", "Ruby",
                     "HTML", "CSS", "TypeScript", "React", "Angular", "Vue.js", "Node.js", "PHP", "Python (Django)", "Laravel", "ASP.NET", "Full-stack",
                     "Android (Java)", "Flutter", "React Native", "iOS (Swift)",
                     "Unity", "Unreal Engine", 
                     "Data Analysis", "Machine Learning", "Deep Learning", "Data Visualization"]
        )
        yield question(
            ident="level",
            Type="multi",
            valid=["Beginner", "Intermediate", "Advanced"],
            text="📈 What is your current skill level?",
            options=["Beginner", "Intermediate", "Advanced"]
        )
        yield question(
            ident="continue_choice",
            Type="multi",
            valid=["yes", "no"],
            text="🔄 Would you like to get another recommendation? (yes/no):",
            options=["yes", "no"]
        )


    @Rule(Fact(start=True), salience=200)
    def start_process(self):
        output = "="*60 + "\n"
        output += "🔎 The expert system is ready to help you.\n"
        output += "Let's get started with your recommendations!\n"
        output += "="*60 
        self.recommend_action(output)
        self.declare(Fact(ask="skill"))


    @Rule(AS.ask << Fact(ask=MATCH.id),
          question(ident=MATCH.id, text=MATCH.text, valid=MATCH.valid, Type=MATCH.Type, options=MATCH.options),
          NOT(Answer(ident=MATCH.id)),
          salience=100)
    def ask_question_by_id(self, ask, id, text, valid, Type, options):
        self.retract(ask)
        answer = self.ask_user(text, Type, valid, options)
        self.declare(Answer(ident=id, text=answer))

    @Rule(Answer(ident="skill", text=MATCH.skill),
          NOT(Fact(ask="sub_skill")),
          NOT(Answer(ident="sub_skill")),
          salience=190)
    def supply_answer_sub_skill(self, skill):
        focus_options = {
            "Game Development": ["Unity", "Unreal Engine"],
            "Data Science": ["Data Analysis", "Machine Learning", "Deep Learning", "Data Visualization"],
            "Mobile App Development": ["Android (Java)", "Flutter", "React Native", "iOS (Swift)"],
            "Web Development": ["HTML", "CSS", "TypeScript", "React", "Angular", "Vue.js",
                                "Node.js", "PHP", "Python (Django)", "Laravel", "ASP.NET", "Full-stack"],
            "Programming": ["Python", "Java", "C++", "JavaScript", "C#", "Ruby"]
        }
        valid_sub_skills = focus_options.get(skill, [])
        if not valid_sub_skills:
            self.recommend_action(f"🙁 Looks like there are no sub-skills available for {skill}. Try picking another skill!")
            self.declare(Fact(ask="skill"))
            return
        sub_skill_fact = [self.facts[f] for f in self.facts if isinstance(self.facts[f], question) and self.facts[f]["ident"] == "sub_skill"][0]
        self.modify(sub_skill_fact,
                    valid=valid_sub_skills,
                    options=valid_sub_skills,
                    text=f"🎯 Awesome! What would you like to focus on in {skill}?")
        self.declare(Fact(ask="sub_skill"))

    @Rule(Answer(ident="sub_skill", text=MATCH.sub_skill),
          Answer(ident="skill", text=MATCH.skill),
          NOT(Fact(ask="level")),
          NOT(Answer(ident="level")),
          salience=180)
    def supply_answer_level(self, sub_skill, skill):
        self.declare(Fact(ask="level"))

    
    # --- Recommendation Rules 📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌 ---

    
    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="Python"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_python_beginner(self):
        details = {
            "ide": {
                "name": "Thonny",
                "url": "https://thonny.org",
                "description": "Beginner-friendly Python IDE with built-in debugger and simple UI.",
                "confidence": 0.85
            },
            "learning_resources": [
                {
                    "name": "W3Schools Python",
                    "type": "Interactive Tutorial",
                    "url": "https://www.w3schools.com/python/",
                    "confidence": 0.9
                },
                {
                    "name": "freeCodeCamp Python Course",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=rfscVS0vtbw",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Python Official Documentation",
                    "url": "https://docs.python.org/3/",
                    "confidence": 0.95
                },
                {
                    "name": "Python Cheatsheet",
                    "url": "https://www.pythoncheatsheet.org/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence)) #ensures the result stays in [-1.0, 1.0]
        self.display_recommendation(details, confidence, "Beginner", "Python", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="Python"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_python_intermediate(self):
        details = {
            "ide": {
                "name": "PyCharm Community",
                "url": "https://www.jetbrains.com/pycharm/",
                "description": "Feature-rich IDE for Python with code analysis and project management.",
                "confidence": 0.90
            },
            "learning_resources": [
                {
                    "name": "Real Python Tutorials",
                    "type": "Interactive Tutorial",
                    "url": "https://realpython.com/",
                    "confidence": 0.85
                },
                {
                    "name": "Corey Schafer Python Tutorials",
                    "type": "YouTube Tutorial",
                    "url": "https://youtube.com/playlist?list=PLprDVKp2F5g15dX01vra7YKEffSPgwljB&si=DW3RIAA-nUdBWCb_",
                    "confidence": 0.8
                }
            ],
            "documentation": [
                {
                    "name": "Python Official Documentation",
                    "url": "https://docs.python.org/3/",
                    "confidence": 0.95
                },
                {
                    "name": "PyCharm Documentation",
                    "url": "https://www.jetbrains.com/help/pycharm/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Python", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="Python"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_python_advanced(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Highly customizable IDE with advanced Python extensions for professional development.",
                "confidence": 0.9
            },
            "learning_resources": [
                {
                    "name": "Advanced Python on Real Python",
                    "type": "Interactive Tutorial",
                    "url": "https://realpython.com/",
                    "confidence": 0.85
                },
                {
                    "name": "Talk Python To Me Courses",
                    "type": "Video Course",
                    "url": "https://training.talkpython.fm/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Python Official Documentation",
                    "url": "https://docs.python.org/3/",
                    "confidence": 0.95
                },
                {
                    "name": "VS Code Python Docs",
                    "url": "https://code.visualstudio.com/docs/languages/python",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Python", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="Java"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_java_beginner(self):
        details = {
            "ide": {
                "name": "Eclipse",
                "url": "https://www.eclipse.org/",
                "description": "Popular open-source IDE for Java with beginner-friendly setup.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Java Tutorial for Beginners",
                    "type": "Interactive Tutorial",
                    "url": "https://www.javatpoint.com/java-tutorial",
                    "confidence": 0.9
                },
                {
                    "name": "Java Programming Course by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=grEKMHGYyns",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Java Official Documentation",
                    "url": "https://docs.oracle.com/en/java/",
                    "confidence": 0.95
                },
                {
                    "name": "Java Cheatsheet",
                    "url": "https://www.codecademy.com/learn/learn-java/modules/learn-java-introduction-to-java/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Java", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="Java"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_java_intermediate(self):
        details = {
            "ide": {
                "name": "IntelliJ IDEA Community",
                "url": "https://www.jetbrains.com/idea/",
                "description": "Powerful Java IDE with code completion and debugging tools.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Baeldung Java Tutorials",
                    "type": "Interactive Tutorial",
                    "url": "https://www.baeldung.com/",
                    "confidence": 0.85
                },
                {
                    "name": "Java Intermediate by Programming with Mosh",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=eIrMbAQSU34",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Java Official Documentation",
                    "url": "https://docs.oracle.com/en/java/",
                    "confidence": 0.95
                },
                {
                    "name": "IntelliJ IDEA Documentation",
                    "url": "https://www.jetbrains.com/help/idea/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Java", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="Java"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_java_advanced(self):
        details = {
            "ide": {
                "name": "IntelliJ IDEA Ultimate",
                "url": "https://www.jetbrains.com/idea/",
                "description": "Advanced IDE with enterprise-level Java features and integrations.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Spring Framework Tutorials",
                    "type": "Interactive Tutorial",
                    "url": "https://spring.io/guides",
                    "confidence": 0.85
                },
                {
                    "name": "Java Advanced Topics by Baeldung",
                    "type": "Article Series",
                    "url": "https://www.baeldung.com/java",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Java Official Documentation",
                    "url": "https://docs.oracle.com/en/java/",
                    "confidence": 0.95
                },
                {
                    "name": "Spring Documentation",
                    "url": "https://docs.spring.io/spring-framework/docs/current/reference/html/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Java", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="C++"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_cpp_beginner(self):
        details = {
            "ide": {
                "name": "Code::Blocks",
                "url": "http://www.codeblocks.org/",
                "description": "Lightweight, open-source IDE for C++ with simple setup for beginners.",
                "confidence": 0.9
            },
            "learning_resources": [
                {
                    "name": "LearnCpp.com",
                    "type": "Interactive Tutorial",
                    "url": "https://www.learncpp.com/",
                    "confidence": 0.9
                },
                {
                    "name": "C++ Tutorial by The Cherno",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=18c3MTX0PK0&list=PLlrATfBNZ98dudnM48yfGUldqGD0S4FFb",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "C++ Reference",
                    "url": "https://en.cppreference.com/w/",
                    "confidence": 0.95
                },
                {
                    "name": "C++ Cheatsheet",
                    "url": "https://www.codecademy.com/learn/learn-c-plus-plus/modules/learn-cpp-introduction/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "C++", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="C++"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_cpp_intermediate(self):
        details = {
            "ide": {
                "name": "CLion",
                "url": "https://www.jetbrains.com/clion/",
                "description": "Robust IDE for C++ with advanced code analysis and debugging.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "C++ Intermediate on LearnCpp",
                    "type": "Interactive Tutorial",
                    "url": "https://www.learncpp.com/",
                    "confidence": 0.9
                },
                {
                    "name": "C++ Programming by Derek Banas",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=Rub-JsjMhWY",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "C++ Reference",
                    "url": "https://en.cppreference.com/w/",
                    "confidence": 0.95
                },
                {
                    "name": "CLion Documentation",
                    "url": "https://www.jetbrains.com/help/clion/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "C++", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="C++"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_cpp_advanced(self):
        details = {
            "ide": {
                "name": "Visual Studio",
                "url": "https://visualstudio.microsoft.com/",
                "description": "Professional IDE with advanced C++ tools for performance-critical applications.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Modern C++ Tutorials",
                    "type": "Article Series",
                    "url": "https://learnmoderncpp.com/",
                    "confidence": 0.85
                },
                {
                    "name": "CppMasteryQuest",
                    "type": "GitHub Repository",
                    "url": "https://github.com/SourenaMOOSAVI/CppMasteryQuest",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "C++ Reference",
                    "url": "https://en.cppreference.com/w/",
                    "confidence": 0.95
                },
                {
                    "name": "Visual Studio C++ Docs",
                    "url": "https://docs.microsoft.com/en-us/cpp/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "C++", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="JavaScript"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_javascript_beginner(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Lightweight, beginner-friendly IDE with JavaScript support.",
                "confidence": 0.9
            },
            "learning_resources": [
                {
                    "name": "W3Schools JavaScript",
                    "type": "Interactive Tutorial",
                    "url": "https://www.w3schools.com/js/",
                    "confidence": 0.9
                },
                {
                    "name": "JavaScript Basics by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=PkZNo7MFoVs",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "MDN JavaScript Docs",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/JavaScript",
                    "confidence": 0.95
                },
                {
                    "name": "JavaScript Cheatsheet",
                    "url": "https://www.codecademy.com/learn/introduction-to-javascript/modules/learn-javascript-introduction/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "JavaScript", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="JavaScript"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_javascript_intermediate(self):
        details = {
            "ide": {
                "name": "WebStorm",
                "url": "https://www.jetbrains.com/webstorm/",
                "description": "Advanced IDE for JavaScript with framework support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "JavaScript30",
                    "type": "Interactive Tutorial",
                    "url": "https://javascript30.com/",
                    "confidence": 0.9
                },
                {
                    "name": "JavaScript Intermediate by SoloLearn",
                    "type": "Online Course",
                    "url": "https://www.sololearn.com/en/learn/courses/javascript-intermediate",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "MDN JavaScript Docs",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/JavaScript",
                    "confidence": 0.95
                },
                {
                    "name": "WebStorm Documentation",
                    "url": "https://www.jetbrains.com/help/webstorm/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "JavaScript", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="JavaScript"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_javascript_advanced(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Customizable IDE with advanced JavaScript and Node.js support.",
                "confidence": 0.9
            },
            "learning_resources": [
                {
                    "name": "You Don’t Know JS",
                    "type": "Book Series",
                    "url": "https://github.com/getify/You-Dont-Know-JS",
                    "confidence": 0.9
                },
                {
                    "name": "JavaScript.info",
                    "type": "Interactive Tutorial",
                    "url": "https://javascript.info/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "MDN JavaScript Docs",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/JavaScript",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "JavaScript", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="C#"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_csharp_beginner(self):
        details = {
            "ide": {
                "name": "Visual Studio Community",
                "url": "https://visualstudio.microsoft.com/",
                "description": "Free, beginner-friendly IDE for C# with built-in tutorials.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "C# Tutorial by Tutorialspoint",
                    "type": "Interactive Tutorial",
                    "url": "https://www.tutorialspoint.com/csharp/",
                    "confidence": 0.9
                },
                {
                    "name": "C# Basics by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=GhQdlIFL_YQ",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "C# Official Documentation",
                    "url": "https://docs.microsoft.com/en-us/dotnet/csharp/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "C#", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="C#"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_csharp_intermediate(self):
        details = {
            "ide": {
                "name": "Visual Studio",
                "url": "https://visualstudio.microsoft.com/",
                "description": "Powerful IDE for C# with debugging and project management.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "C# Intermediate on Pluralsight",
                    "type": "Video Course",
                    "url": "https://www.pluralsight.com/paths/csharp",
                    "confidence": 0.9
                },
                {
                    "name": "C# Tutorials by Mosh Hamedani",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/playlist?list=PLTjRvopDNEzS3sL7azL-M411_D2sF2M8A",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "C# Official Documentation",
                    "url": "https://docs.microsoft.com/en-us/dotnet/csharp/",
                    "confidence": 0.95
                },
                {
                    "name": ".NET Documentation",
                    "url": "https://docs.microsoft.com/en-us/dotnet/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "C#", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="C#"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_csharp_advanced(self):
        details = {
            "ide": {
                "name": "Rider",
                "url": "https://www.jetbrains.com/rider/",
                "description": "Cross-platform IDE for advanced C# development with .NET support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced C# on Pluralsight",
                    "type": "Video Course",
                    "url": "https://www.pluralsight.com/paths/csharp",
                    "confidence": 0.9
                },
                {
                    "name": "C# Advanced Topics",
                    "type": "Article Series",
                    "url": "https://www.c-sharpcorner.com/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "C# Official Documentation",
                    "url": "https://docs.microsoft.com/en-us/dotnet/csharp/",
                    "confidence": 0.95
                },
                {
                    "name": "Rider Documentation",
                    "url": "https://www.jetbrains.com/help/rider/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "C#", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
      Answer(ident="sub_skill", text="HTML"),
      Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_html_beginner(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Lightweight, beginner-friendly IDE with HTML extensions and live preview.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "W3Schools HTML Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://www.w3schools.com/html/",
                    "confidence": 0.9
                },
                {
                    "name": "freeCodeCamp HTML Course",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=pQN-pnXPaVg",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "MDN HTML Documentation",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/HTML",
                    "confidence": 0.95
                },
                {
                    "name": "HTML Cheatsheet",
                    "url": "https://www.codecademy.com/learn/learn-html/modules/learn-html-elements/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "HTML", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="HTML"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_html_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Customizable IDE with HTML5 and accessibility-focused extensions.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "MDN HTML Intermediate",
                    "type": "Interactive Tutorial",
                    "url": "https://developer.mozilla.org/en-US/docs/Learn/HTML",
                    "confidence": 0.95
                },
                {
                    "name": "HTML5 Advanced by Traversy Media",
                    "type": "YouTube Tutorial",
                    "url": "https://youtu.be/Wm6CUkswsNw?si=FebLYX2Mp7pjb0m-",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "MDN HTML Documentation",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/HTML",
                    "confidence": 0.95
                },
                {
                    "name": "HTML5 Specification",
                    "url": "https://html.spec.whatwg.org/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "HTML", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))


    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="HTML"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_html_advanced(self):
        details = {
            "ide": {
                "name": "WebStorm",
                "url": "https://www.jetbrains.com/webstorm/",
                "description": "Professional IDE with advanced HTML tools for accessibility and SEO.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Web Accessibility by Google",
                    "type": "Interactive Tutorial",
                    "url": "https://web.dev/learn/accessibility/",
                    "confidence": 0.9
                },
                {
                    "name": "Advanced HTML Semantics",
                    "type": "Article Series",
                    "url": "https://www.smashingmagazine.com/category/html/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "MDN HTML Documentation",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/HTML",
                    "confidence": 0.95
                },
                {
                    "name": "Web Accessibility Guidelines",
                    "url": "https://www.w3.org/WAI/standards-guidelines/wcag/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "HTML", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
        
        # ✅ Ruby - Beginner
    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="Ruby"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_ruby_beginner(self):
        details = {
            "ide": {
                "name": "RubyMine",
                "url": "https://www.jetbrains.com/rubymine/",
                "description": "User-friendly IDE for Ruby with beginner-oriented features.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Learn Ruby - Codecademy",
                    "type": "Interactive Tutorial",
                    "url": "https://www.codecademy.com/learn/learn-ruby",
                    "confidence": 0.9
                },
                {
                    "name": "Ruby Tutorial",
                    "type": "YouTube Tutorial",
                    "url": "https://youtube.com/playlist?list=PL3mtAHT_eRex2wgl-cYdKcYoYcLNK7J8A",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Ruby Official Documentation",
                    "url": "https://www.ruby-lang.org/en/documentation/",
                    "confidence": 0.95
                },
                {
                    "name": "Ruby Cheatsheet",
                    "url": "https://www.codecademy.com/learn/learn-ruby/modules/learn-ruby-introduction/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Beginner", "Ruby", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    # ✅ Ruby - Intermediate
    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="Ruby"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_ruby_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Customizable IDE with Ruby extensions for intermediate developers.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Ruby on Rails Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://www.railstutorial.org/",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Ruby Official Documentation",
                    "url": "https://www.ruby-lang.org/en/documentation/",
                    "confidence": 0.95
                },
                {
                    "name": "Rails Documentation",
                    "url": "https://guides.rubyonrails.org/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Intermediate", "Ruby", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    # ✅ Ruby - Advanced
    @Rule(Answer(ident="skill", text="Programming"),
          Answer(ident="sub_skill", text="Ruby"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_ruby_advanced(self):
        details = {
            "ide": {
                "name": "RubyMine",
                "url": "https://www.jetbrains.com/rubymine/",
                "description": "Advanced IDE with Ruby on Rails and debugging support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced Ruby on GoRails",
                    "type": "Video Course",
                    "url": "https://gorails.com/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Ruby Official Documentation",
                    "url": "https://www.ruby-lang.org/en/documentation/",
                    "confidence": 0.95
                },
                {
                    "name": "RubyMine Documentation",
                    "url": "https://www.jetbrains.com/help/rubymine/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Advanced", "Ruby", "Programming")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    # ✅ CSS - Beginner
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="CSS"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_css_beginner(self):
        details = {
            "ide": {
                "name": "CodePen",
                "url": "https://codepen.io/",
                "description": "Online IDE for experimenting with CSS, ideal for beginners.",
                "confidence": 0.9
            },
            "learning_resources": [
                {
                    "name": "W3Schools CSS Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://www.w3schools.com/css/",
                    "confidence": 0.9
                },
                {
                    "name": "CSS Crash Course by Traversy Media",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=yfoY53QXEnI",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "MDN CSS Documentation",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/CSS",
                    "confidence": 0.95
                },
                {
                    "name": "CSS Cheatsheet",
                    "url": "https://www.codecademy.com/learn/learn-css/modules/learn-css-selectors-visual-rules/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Beginner", "CSS", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    # ✅ CSS - Intermediate
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="CSS"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_css_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Versatile IDE with CSS extensions for animations and frameworks.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "CSS Tricks Guides",
                    "type": "Article Series",
                    "url": "https://css-tricks.com/",
                    "confidence": 0.9
                },
                {
                    "name": "CSS Tutorial",
                    "type": "YouTube Tutorial",
                    "url": "https://youtu.be/0hrJGWrCux0?si=qnUOVAYQJnihvkXD",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "MDN CSS Documentation",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/CSS",
                    "confidence": 0.95
                },
                {
                    "name": "CSS Grid Documentation",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/CSS/CSS_Grid_Layout",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Intermediate", "CSS", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    # ✅ CSS - Advanced
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="CSS"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_css_advanced(self):
        details = {
            "ide": {
                "name": "WebStorm",
                "url": "https://www.jetbrains.com/webstorm/",
                "description": "Professional IDE with advanced CSS and preprocessor support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced CSS and Sass",
                    "type": "Video Course",
                    "url": "https://www.udemy.com/course/advanced-css-and-sass/",
                    "confidence": 0.9
                },
                {
                    "name": "CSS Architecture by Smashing Magazine",
                    "type": "Article Series",
                    "url": "https://www.smashingmagazine.com/category/css/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "MDN CSS Documentation",
                    "url": "https://developer.mozilla.org/en-US/docs/Web/CSS",
                    "confidence": 0.95
                },
                {
                    "name": "Tailwind CSS Documentation",
                    "url": "https://tailwindcss.com/docs",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Advanced", "CSS", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))



    @Rule(Answer(ident="skill", text="Web Development"),
      Answer(ident="sub_skill", text="TypeScript"),
      Answer(ident="level", text="Beginner"),
      salience=1000)
    def recommend_typescript_beginner(self):
        details = {
            "ide": {
                "name": "CodeSandbox",
                "url": "https://codesandbox.io/",
                "description": "Online IDE for TypeScript, ideal for beginners with no setup required.",
                "confidence": 0.9
            },
            "learning_resources": [
                {
                    "name": "TypeScript Official Handbook",
                    "type": "Interactive Tutorial",
                    "url": "https://www.typescriptlang.org/docs/handbook/intro.html",
                    "confidence": 0.95
                },
                {
                    "name": "TypeScript Basics by The Net Ninja",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/playlist?list=PL4cUxeGkcC9gUgr39Q_yD6v-bSyMwKPUI",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "TypeScript Official Documentation",
                    "url": "https://www.typescriptlang.org/docs/",
                    "confidence": 0.95
                },
                {
                    "name": "TypeScript Cheatsheet",
                    "url": "https://www.codecademy.com/learn/learn-typescript/modules/learn-typescript-introduction/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "TypeScript", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    # نفس النمط للمستويين Intermediate و Advanced
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="TypeScript"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_typescript_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Versatile IDE with TypeScript support for front-end frameworks.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "TypeScript Deep Dive",
                    "type": "Interactive Tutorial",
                    "url": "https://basarat.gitbook.io/typescript/",
                    "confidence": 0.9
                },
                {
                    "name": "TypeScript with React",
                    "type": "YouTube Tutorial",
                    "url": "https://youtu.be/TPACABQTHvM?si=wnAew0agnaJuhMzk",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "TypeScript Official Documentation",
                    "url": "https://www.typescriptlang.org/docs/",
                    "confidence": 0.95
                },
                {
                    "name": "React with TypeScript Docs",
                    "url": "https://react.dev/learn/typescript",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "TypeScript", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="TypeScript"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_typescript_advanced(self):
        details = {
            "ide": {
                "name": "WebStorm",
                "url": "https://www.jetbrains.com/webstorm/",
                "description": "Professional IDE with advanced TypeScript and framework support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced TypeScript by TypeScript Training",
                    "type": "Video Course",
                    "url": "https://www.typescript-training.com/",
                    "confidence": 0.85
                },
                {
                    "name": "TypeScript Patterns",
                    "type": "Article Series",
                    "url": "https://refactoring.guru/design-patterns/typescript",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "TypeScript Official Documentation",
                    "url": "https://www.typescriptlang.org/docs/",
                    "confidence": 0.95
                },
                {
                    "name": "TypeScript Advanced Types Docs",
                    "url": "https://www.typescriptlang.org/docs/handbook/advanced-types.html",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "TypeScript", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="React"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_react_beginner(self):
        details = {
            "ide": {
                "name": "CodeSandbox",
                "url": "https://codesandbox.io/",
                "description": "Online IDE for React, perfect for beginners with no setup required.",
                "confidence": 0.9
            },
            "learning_resources": [
                {
                    "name": "React Official Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://react.dev/learn",
                    "confidence": 0.95
                },
                {
                    "name": "React Basics by Scrimba",
                    "type": "Interactive Course",
                    "url": "https://scrimba.com/learn/learnreact",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "React Official Documentation",
                    "url": "https://react.dev/",
                    "confidence": 0.95
                },
                {
                    "name": "React Cheatsheet",
                    "url": "https://devhints.io/react",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "React", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="React"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_react_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Popular IDE with React extensions for component-based development.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "React Official Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://react.dev/learn",
                    "confidence": 0.95
                },
                {
                    "name": "React Course by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=bMknfKXIFA8",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "React Official Documentation",
                    "url": "https://react.dev/",
                    "confidence": 0.95
                },
                {
                    "name": "React Cheatsheet",
                    "url": "https://devhints.io/react",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "React", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="React"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_react_advanced(self):
        details = {
            "ide": {
                "name": "WebStorm",
                "url": "https://www.jetbrains.com/webstorm/",
                "description": "Professional IDE with advanced React and TypeScript support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced React by Frontend Masters",
                    "type": "Video Course",
                    "url": "https://frontendmasters.com/learn/react/",
                    "confidence": 0.9
                },
                {
                    "name": "React Patterns by React Training",
                    "type": "Article Series",
                    "url": "https://reacttraining.com/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "React Official Documentation",
                    "url": "https://react.dev/",
                    "confidence": 0.95
                },
                {
                    "name": "React Hooks Documentation",
                    "url": "https://react.dev/reference/react",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "React", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
        
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Angular"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_angular_beginner(self):
        details = {
            "ide": {
                "name": "StackBlitz",
                "url": "https://stackblitz.com/",
                "description": "Online IDE for Angular, ideal for beginners with instant setup.",
                "confidence": 0.9
            },
            "learning_resources": [
                {"name": "Angular Getting Started", "type": "Interactive Tutorial", "url": "https://angular.io/start", "confidence": 0.95},
                {"name": "Angular Basics by Codevolution", "type": "YouTube Tutorial", "url": "https://www.youtube.com/playlist?list=PLC3y8-rFHvwhBRAgFinJR8KHIrCdTkZcZ", "confidence": 0.85}
            ],
            "documentation": [
                {"name": "Angular Official Documentation", "url": "https://angular.io/docs", "confidence": 0.95},
                {"name": "Angular Cheatsheet", "url": "https://angular.io/guide/cheatsheet", "confidence": 0.9}
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]: confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]: confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Beginner", "Angular", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Angular"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_angular_intermediate(self):
        details = {
            "ide": {"name": "VS Code", "url": "https://code.visualstudio.com/", "description": "Versatile IDE with Angular CLI and TypeScript support.", "confidence": 0.95},
            "learning_resources": [
                {"name": "Angular Tour of Heroes", "type": "Interactive Tutorial", "url": "https://angular.io/tutorial", "confidence": 0.95},
                {"name": "Angular Course by Maximilian Schwarzmüller", "type": "Video Course", "url": "https://www.udemy.com/course/the-complete-guide-to-angular-2/", "confidence": 0.9}
            ],
            "documentation": [
                {"name": "Angular Official Documentation", "url": "https://angular.io/docs", "confidence": 0.95},
                {"name": "Angular Cheatsheet", "url": "https://angular.io/guide/cheatsheet", "confidence": 0.9}
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]: confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]: confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Intermediate", "Angular", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Angular"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_angular_advanced(self):
        details = {
            "ide": {"name": "WebStorm", "url": "https://www.jetbrains.com/webstorm/", "description": "Advanced IDE with Angular-specific tools and debugging.", "confidence": 0.95},
            "learning_resources": [
                {"name": "Advanced Angular by Angular University", "type": "Video Course", "url": "https://angular-university.io/", "confidence": 0.9},
                {"name": "Angular In-Depth", "type": "Article Series", "url": "https://angular.love/", "confidence": 0.85}
            ],
            "documentation": [
                {"name": "Angular Official Documentation", "url": "https://angular.io/docs", "confidence": 0.95},
                {"name": "Angular CLI Documentation", "url": "https://angular.io/cli", "confidence": 0.9}
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]: confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]: confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Advanced", "Angular", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Vue.js"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_vue_beginner(self):
        details = {
            "ide": {"name": "CodeSandbox", "url": "https://codesandbox.io/", "description": "Online IDE for Vue.js, perfect for beginners with no setup.", "confidence": 0.9},
            "learning_resources": [
                {"name": "Vue.js Quick Start", "type": "Interactive Tutorial", "url": "https://vuejs.org/guide/quick-start.html", "confidence": 0.95},
                {"name": "Vue.js Basics by The Net Ninja", "type": "YouTube Tutorial", "url": "https://youtube.com/playlist?list=PL4cUxeGkcC9hYYGbV60Vq3IXYNfDk8At1&si=IwWAQM6r41mjEMJ3", "confidence": 0.85}
            ],
            "documentation": [
                {"name": "Vue.js Official Documentation", "url": "https://vuejs.org/guide/introduction.html", "confidence": 0.95},
                {"name": "Vue.js Cheatsheet", "url": "https://devhints.io/vue", "confidence": 0.9}
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]: confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]: confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Beginner", "Vue.js", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Vue.js"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_vue_intermediate(self):
        details = {
            "ide": {"name": "VS Code", "url": "https://code.visualstudio.com/", "description": "Flexible IDE with Vue.js extensions for component development.", "confidence": 0.95},
            "learning_resources": [
                {"name": "Vue.js Official Tutorial", "type": "Interactive Tutorial", "url": "https://vuejs.org/guide/introduction.html", "confidence": 0.95}
            ],
            "documentation": [
                {"name": "Vue.js Official Documentation", "url": "https://vuejs.org/guide/introduction.html", "confidence": 0.95},
                {"name": "Vue.js Cheatsheet", "url": "https://devhints.io/vue", "confidence": 0.9}
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]: confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]: confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Intermediate", "Vue.js", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Vue.js"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_vue_advanced(self):
        details = {
            "ide": {"name": "WebStorm", "url": "https://www.jetbrains.com/webstorm/", "description": "Professional IDE with Vue.js and Nuxt.js support.", "confidence": 0.95},
            "learning_resources": [
                {"name": "Vue Mastery Advanced", "type": "Video Course", "url": "https://www.vuemastery.com/courses/", "confidence": 0.9}
            ],
            "documentation": [
                {"name": "Vue.js Official Documentation", "url": "https://vuejs.org/guide/introduction.html", "confidence": 0.95},
                {"name": "Nuxt.js Documentation", "url": "https://nuxt.com/docs", "confidence": 0.9}
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]: confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]: confidence = self.combine_confidence(confidence, d["confidence"])
        self.display_recommendation(details, confidence, "Advanced", "Vue.js", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Python (Django)"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_django_beginner(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Free IDE with Python and Django extensions for beginners.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Django Girls Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://tutorial.djangogirls.org/",
                    "confidence": 0.9
                },
                {
                    "name": "Django Basics",
                    "type": "YouTube Tutorial",
                    "url": "https://youtu.be/rHux0gMZ3Eg?si=MdDkhoMCF0-FGHQE",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Django Official Documentation",
                    "url": "https://docs.djangoproject.com/en/stable/",
                    "confidence": 0.95
                },
                {
                    "name": "Python Official Documentation",
                    "url": "https://docs.python.org/3/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for res in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, res["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Python (Django)", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Python (Django)"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_django_intermediate(self):
        details = {
            "ide": {
                "name": "PyCharm Community",
                "url": "https://www.jetbrains.com/pycharm/",
                "description": "Free IDE with Django support for Python web development.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Django Official Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://docs.djangoproject.com/en/stable/intro/tutorial01/",
                    "confidence": 0.95
                }
            ],
            "documentation": [
                {
                    "name": "Django Official Documentation",
                    "url": "https://docs.djangoproject.com/en/stable/",
                    "confidence": 0.95
                },
                {
                    "name": "Python Official Documentation",
                    "url": "https://docs.python.org/3/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for res in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, res["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Python (Django)", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Python (Django)"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_django_advanced(self):
        details = {
            "ide": {
                "name": "PyCharm Professional",
                "url": "https://www.jetbrains.com/pycharm/",
                "description": "Advanced IDE with Django and database integration.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced Django by Real Python",
                    "type": "Article Series",
                    "url": "https://realpython.com/tutorials/django/",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Django Official Documentation",
                    "url": "https://docs.djangoproject.com/en/stable/",
                    "confidence": 0.95
                },
                {
                    "name": "Django REST Framework Docs",
                    "url": "https://www.django-rest-framework.org/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for res in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, res["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Python (Django)", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Laravel"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_laravel_beginner(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Free IDE with PHP and Laravel extensions for beginners.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Laravel From Scratch",
                    "type": "Interactive Tutorial",
                    "url": "https://laracasts.com/series/laravel-8-from-scratch",
                    "confidence": 0.9
                },
                {
                    "name": "Laravel Basics by Traversy Media",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=ImtZ5yENzgE",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Laravel Official Documentation",
                    "url": "https://laravel.com/docs",
                    "confidence": 0.95
                },
                {
                    "name": "PHP Official Documentation",
                    "url": "https://www.php.net/docs.php",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for res in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, res["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Laravel", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Laravel"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_laravel_intermediate(self):
        details = {
            "ide": {
                "name": "PhpStorm",
                "url": "https://www.jetbrains.com/phpstorm/",
                "description": "Professional IDE with Laravel-specific tools and debugging.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Laravel Official Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://laravel.com/docs",
                    "confidence": 0.95
                }
            ],
            "documentation": [
                {
                    "name": "Laravel Official Documentation",
                    "url": "https://laravel.com/docs",
                    "confidence": 0.95
                },
                {
                    "name": "Laravel API Documentation",
                    "url": "https://laravel.com/api",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for res in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, res["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Laravel", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Laravel"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_laravel_advanced(self):
        details = {
            "ide": {
                "name": "PhpStorm",
                "url": "https://www.jetbrains.com/phpstorm/",
                "description": "Advanced IDE with Laravel performance optimization tools.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Laravel Patterns",
                    "type": "Article Series",
                    "url": "https://laravel-news.com/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Laravel Official Documentation",
                    "url": "https://laravel.com/docs",
                    "confidence": 0.95
                },
                {
                    "name": "Laravel Vapor Documentation",
                    "url": "https://vapor.laravel.com/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for res in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, res["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Laravel", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="ASP.NET"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_aspnet_beginner(self):
        details = {
            "ide": {
                "name": "Visual Studio Code",
                "url": "https://code.visualstudio.com/",
                "description": "Free IDE with C# and ASP.NET extensions for beginners.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "ASP.NET Basics by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=C5cnZ-gZy2I",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "ASP.NET Core Documentation",
                    "url": "https://learn.microsoft.com/en-us/aspnet/core/",
                    "confidence": 0.95
                },
                {
                    "name": "C# Official Documentation",
                    "url": "https://learn.microsoft.com/en-us/dotnet/csharp/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for res in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, res["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "ASP.NET", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="ASP.NET"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_aspnet_intermediate(self):
        details = {
            "ide": {
                "name": "Visual Studio Community",
                "url": "https://visualstudio.microsoft.com/",
                "description": "Free IDE with ASP.NET Core support for web development.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "ASP.NET Core MVC Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://learn.microsoft.com/en-us/aspnet/core/tutorials/first-mvc-app/",
                    "confidence": 0.95
                }
            ],
            "documentation": [
                {
                    "name": "ASP.NET Core Documentation",
                    "url": "https://learn.microsoft.com/en-us/aspnet/core/",
                    "confidence": 0.95
                },
                {
                    "name": ".NET Documentation",
                    "url": "https://learn.microsoft.com/en-us/dotnet/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for res in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, res["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "ASP.NET", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="ASP.NET"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_aspnet_advanced(self):
        details = {
            "ide": {
                "name": "Visual Studio",
                "url": "https://visualstudio.microsoft.com/",
                "description": "Professional IDE with advanced ASP.NET Core and Blazor support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Blazor Tutorial",
                    "type": "Article Series",
                    "url": "https://learn.microsoft.com/en-us/aspnet/core/blazor/",
                    "confidence": 0.95
                }
            ],
            "documentation": [
                {
                    "name": "ASP.NET Core Documentation",
                    "url": "https://learn.microsoft.com/en-us/aspnet/core/",
                    "confidence": 0.95
                },
                {
                    "name": "Blazor Documentation",
                    "url": "https://learn.microsoft.com/en-us/aspnet/core/blazor/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for res in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, res["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "ASP.NET", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Full-stack"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_fullstack_beginner(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Free IDE with HTML, CSS, and JavaScript support for beginners.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "The Odin Project",
                    "type": "Interactive Tutorial",
                    "url": "https://www.theodinproject.com/paths/full-stack-javascript",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "MDN Web Docs",
                    "url": "https://developer.mozilla.org/en-US/",
                    "confidence": 0.95
                },
                {
                    "name": "JavaScript Cheatsheet",
                    "url": "https://www.codecademy.com/learn/introduction-to-javascript/modules/learn-javascript-introduction/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Full-stack", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Full-stack"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_fullstack_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Versatile IDE for full-stack with React and Node.js support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Full Stack Open",
                    "type": "Interactive Tutorial",
                    "url": "https://fullstackopen.com/",
                    "confidence": 0.9
                },
                {
                    "name": "MERN Stack Course by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=7CqJlxBYj-M",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "React Official Documentation",
                    "url": "https://react.dev/",
                    "confidence": 0.95
                },
                {
                    "name": "Node.js Official Documentation",
                    "url": "https://nodejs.org/en/docs/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Full-stack", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Web Development"),
          Answer(ident="sub_skill", text="Full-stack"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_fullstack_advanced(self):
        details = {
            "ide": {
                "name": "WebStorm",
                "url": "https://www.jetbrains.com/webstorm/",
                "description": "Advanced IDE for full-stack with React, Node.js, and database tools.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced Full Stack by Frontend Masters",
                    "type": "Video Course",
                    "url": "https://frontendmasters.com/courses/?q=fullstack&c=courses&dg=1",
                    "confidence": 0.9
                },
                {
                    "name": "Full Stack Patterns",
                    "type": "Article Series",
                    "url": "https://www.patterns.dev/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Next.js Documentation",
                    "url": "https://nextjs.org/docs",
                    "confidence": 0.95
                },
                {
                    "name": "Express.js Documentation",
                    "url": "https://expressjs.com/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Full-stack", "Web Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))


    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="Android (Java)"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_android_java_beginner(self):
        details = {
            "ide": {
                "name": "Android Studio",
                "url": "https://developer.android.com/studio",
                "description": "Official IDE for Android development with Java, beginner-friendly with templates.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Android Java Tutorial by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=fis26HvvDII",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Android Developer Documentation",
                    "url": "https://developer.android.com/docs",
                    "confidence": 0.95
                },
                {
                    "name": "Java for Android Cheatsheet",
                    "url": "https://www.codecademy.com/learn/learn-java/modules/learn-java-introduction/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Android (Java)", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="Android (Java)"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_android_java_intermediate(self):
        details = {
            "ide": {
                "name": "Android Studio",
                "url": "https://developer.android.com/studio",
                "description": "Robust IDE with Java support for building complex Android apps.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Android Java",
                    "type": "YouTube Tutorial",
                    "url": "https://youtu.be/tZvjSl9dswg?si=eITgE-3NqVLDXs6Q",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Android Developer Documentation",
                    "url": "https://developer.android.com/docs",
                    "confidence": 0.95
                },
                {
                    "name": "Android API Reference",
                    "url": "https://developer.android.com/reference",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Android (Java)", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="Android (Java)"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_android_java_advanced(self):
        details = {
            "ide": {
                "name": "Android Studio",
                "url": "https://developer.android.com/studio",
                "description": "Professional IDE with advanced Java tools for performance optimization.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Android Architecture Patterns",
                    "type": "Article Series",
                    "url": "https://www.raywenderlich.com/android",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Android Developer Documentation",
                    "url": "https://developer.android.com/docs",
                    "confidence": 0.95
                },
                {
                    "name": "Jetpack Libraries Documentation",
                    "url": "https://developer.android.com/jetpack",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Android (Java)", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="iOS (Swift)"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_ios_swift_beginner(self):
        details = {
            "ide": {
                "name": "Xcode",
                "url": "https://developer.apple.com/xcode/",
                "description": "Official IDE for iOS development with Swift, beginner-friendly with templates.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Swift Playgrounds",
                    "type": "Interactive Tutorial",
                    "url": "https://www.apple.com/swift/playgrounds/",
                    "confidence": 0.95
                }
            ],
            "documentation": [
                {
                    "name": "Swift Documentation",
                    "url": "https://swift.org/documentation/",
                    "confidence": 0.95
                },
                {
                    "name": "Apple Developer Documentation",
                    "url": "https://developer.apple.com/documentation/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "iOS (Swift)", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="iOS (Swift)"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_ios_swift_intermediate(self):
        details = {
            "ide": {
                "name": "Xcode",
                "url": "https://developer.apple.com/xcode/",
                "description": "Robust IDE for Swift with UIKit and SwiftUI support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Developing iOS Apps with Swift",
                    "type": "Interactive Tutorial",
                    "url": "https://developer.apple.com/tutorials/swiftui",
                    "confidence": 0.95
                }
            ],
            "documentation": [
                {
                    "name": "SwiftUI Documentation",
                    "url": "https://developer.apple.com/documentation/swiftui",
                    "confidence": 0.95
                },
                {
                    "name": "Apple Developer Documentation",
                    "url": "https://developer.apple.com/documentation/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "iOS (Swift)", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="iOS (Swift)"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_ios_swift_advanced(self):
        details = {
            "ide": {
                "name": "Xcode",
                "url": "https://developer.apple.com/xcode/",
                "description": "Professional IDE with Swift and Combine for advanced iOS apps.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced SwiftUI",
                    "type": "Interactive Tutorial",
                    "url": "https://developer.apple.com/tutorials/swiftui",
                    "confidence": 0.95
                },
                {
                    "name": "Hacking with Swift",
                    "type": "Article Series",
                    "url": "https://www.hackingwithswift.com/",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "SwiftUI Documentation",
                    "url": "https://developer.apple.com/documentation/swiftui",
                    "confidence": 0.95
                },
                {
                    "name": "Combine Framework Documentation",
                    "url": "https://developer.apple.com/documentation/combine",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "iOS (Swift)", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="Flutter"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_flutter_beginner(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Lightweight IDE with Flutter and Dart extensions, ideal for beginners.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Flutter Get Started",
                    "type": "Interactive Tutorial",
                    "url": "https://flutter.dev/docs/get-started/codelab",
                    "confidence": 0.9
                },
                {
                    "name": "Flutter for Beginners by The Net Ninja",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/playlist?list=PL4cUxeGkcC9jLYyp2Aoh6hcWuxFDX6PBJ",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Flutter Documentation",
                    "url": "https://docs.flutter.dev/",
                    "confidence": 0.95
                },
                {
                    "name": "Dart Documentation",
                    "url": "https://dart.dev/guides",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Flutter", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="Flutter"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_flutter_intermediate(self):
        details = {
            "ide": {
                "name": "Android Studio",
                "url": "https://developer.android.com/studio",
                "description": "Robust IDE with Flutter support for cross-platform development.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Flutter Cookbook",
                    "type": "Interactive Tutorial",
                    "url": "https://docs.flutter.dev/cookbook",
                    "confidence": 0.9
                },
                {
                    "name": "Flutter Tutorial",
                    "type": "YouTube Tutorial",
                    "url": "https://youtube.com/playlist?list=PL3wGb9_yWsvKdT0VWQ5AmsdmqctpV7TfK&si=PJ1NE7lY_QL7p4_G",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Flutter Documentation",
                    "url": "https://docs.flutter.dev/",
                    "confidence": 0.95
                },
                {
                    "name": "Flutter Widget Catalog",
                    "url": "https://docs.flutter.dev/development/ui/widgets",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Flutter", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="Flutter"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_flutter_advanced(self):
        details = {
            "ide": {
                "name": "Android Studio",
                "url": "https://developer.android.com/studio",
                "description": "Professional IDE with Flutter for advanced cross-platform apps.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Flutter Architecture",
                    "type": "Article Series",
                    "url": "https://flutter.dev/docs/resources/architectural-overview",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Flutter Documentation",
                    "url": "https://docs.flutter.dev/",
                    "confidence": 0.95
                },
                {
                    "name": "Flutter Platform Integration",
                    "url": "https://docs.flutter.dev/development/platform-integration",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Flutter", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="React Native"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_react_native_beginner(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Lightweight IDE with React Native extensions, ideal for beginners.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "React Native Getting Started",
                    "type": "Interactive Tutorial",
                    "url": "https://reactnative.dev/docs/getting-started",
                    "confidence": 0.9
                },
                {
                    "name": "React Native for Beginners",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=0-S5a0eXPoc",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "React Native Documentation",
                    "url": "https://reactnative.dev/docs",
                    "confidence": 0.95
                },
                {
                    "name": "React Documentation",
                    "url": "https://react.dev/",
                    "confidence": 0.95
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "React Native", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="React Native"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_react_native_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Versatile IDE with React Native debugging and Expo support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "React Native Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://reactnative.dev/docs/tutorial",
                    "confidence": 0.9
                },
                {
                    "name": "React Native by Maximilian Schwarzmüller",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=9JpT4l-2LlY&t=2s",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "React Native Documentation",
                    "url": "https://reactnative.dev/docs",
                    "confidence": 0.95
                },
                {
                    "name": "Expo Documentation",
                    "url": "https://docs.expo.dev/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "React Native", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Mobile App Development"),
          Answer(ident="sub_skill", text="React Native"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_react_native_advanced(self):
        details = {
            "ide": {
                "name": "WebStorm",
                "url": "https://www.jetbrains.com/webstorm/",
                "description": "Professional IDE with React Native and native integration support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced React Native",
                    "type": "Video Course",
                    "url": "https://www.udemy.com/course/react-native-advanced/",
                    "confidence": 0.9
                },
                {
                    "name": "React Native Patterns",
                    "type": "Article Series",
                    "url": "https://www.reactnative.express/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "React Native Documentation",
                    "url": "https://reactnative.dev/docs",
                    "confidence": 0.95
                },
                {
                    "name": "React Native Native Modules",
                    "url": "https://reactnative.dev/docs/native-modules-ios",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "React Native", "Mobile App Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Game Development"),
          Answer(ident="sub_skill", text="Unity"),
          Answer(ident="level", text="Beginner"),
          salience=1000)
    def recommend_unity_beginner(self):
        details = {
            "ide": {
                "name": "Unity Editor",
                "url": "https://unity.com/download",
                "description": "Official IDE for Unity game development, beginner-friendly with 2D/3D templates.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Unity Learn: Beginner Fundamentals",
                    "type": "Interactive Tutorial",
                    "url": "https://learn.unity.com/pathway/junior-programmer",
                    "confidence": 0.9
                },
                {
                    "name": "Unity for Beginners by Brackeys",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/playlist?list=PLPV2KyIb3jR5QFsefuO2RlAgWEz6EvVi6",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Unity Documentation",
                    "url": "https://docs.unity3d.com/Manual/index.html",
                    "confidence": 0.95
                },
                {
                    "name": "Unity C# Scripting Basics",
                    "url": "https://docs.unity3d.com/ScriptReference/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Unity", "Game Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Game Development"),
          Answer(ident="sub_skill", text="Unity"),
          Answer(ident="level", text="Intermediate"),
          salience=1000)
    def recommend_unity_intermediate(self):
        details = {
            "ide": {
                "name": "Unity Editor",
                "url": "https://unity.com/download",
                "description": "Robust IDE for Unity with C# scripting and asset pipeline support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Unity Game Dev by Code Monkey",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/watch?v=AmGSEH7QcDg",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Unity Documentation",
                    "url": "https://docs.unity3d.com/Manual/index.html",
                    "confidence": 0.95
                },
                {
                    "name": "Unity API Reference",
                    "url": "https://docs.unity3d.com/ScriptReference/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Unity", "Game Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Game Development"),
          Answer(ident="sub_skill", text="Unity"),
          Answer(ident="level", text="Advanced"),
          salience=1000)
    def recommend_unity_advanced(self):
        details = {
            "ide": {
                "name": "Rider",
                "url": "https://www.jetbrains.com/rider/",
                "description": "Advanced IDE for Unity C# scripting with performance optimization tools.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Unity DOTS Fundamentals",
                    "type": "Interactive Tutorial",
                    "url": "https://learn.unity.com/course/introduction-to-dots",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Unity Documentation",
                    "url": "https://docs.unity3d.com/Manual/index.html",
                    "confidence": 0.95
                },
                {
                    "name": "Unity DOTS Documentation",
                    "url": "https://docs.unity3d.com/Packages/com.unity.entities@latest",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for r in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, r["confidence"])
        for d in details["documentation"]:
            confidence = self.combine_confidence(confidence, d["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Unity", "Game Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    
    @Rule(
        Answer(ident="skill", text="Game Development"),
        Answer(ident="sub_skill", text="Unreal Engine"),
        Answer(ident="level", text="Beginner"),
        salience=1000
    )
    def recommend_unreal_beginner(self):
        details = {
            "ide": {
                "name": "Unreal Editor",
                "url": "https://www.unrealengine.com/en-US/download",
                "description": "Official IDE for Unreal Engine, beginner-friendly with Blueprints visual scripting.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Unreal Online Learning: First Steps",
                    "type": "Interactive Tutorial",
                    "url": "https://learn.unrealengine.com/course/2504329",
                    "confidence": 0.9
                },
                {
                    "name": "Unreal Engine for Beginners by Unreal Sensei",
                    "type": "YouTube Tutorial",
                    "url": "https://youtu.be/k-zMkzmduqI?si=Mb5Sf7HOR1fws15F",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Unreal Engine Documentation",
                    "url": "https://docs.unrealengine.com/",
                    "confidence": 0.95
                },
                {
                    "name": "Blueprints Visual Scripting",
                    "url": "https://docs.unrealengine.com/en-US/Engine/Blueprints/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Unreal Engine", "Game Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    
    @Rule(
        Answer(ident="skill", text="Game Development"),
        Answer(ident="sub_skill", text="Unreal Engine"),
        Answer(ident="level", text="Intermediate"),
        salience=1000
    )
    def recommend_unreal_intermediate(self):
        details = {
            "ide": {
                "name": "Unreal Editor",
                "url": "https://www.unrealengine.com/en-US/download",
                "description": "Robust IDE for Unreal with Blueprints and C++ scripting support.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Unreal Online Learning: Intermediate",
                    "type": "Interactive Tutorial",
                    "url": "https://learn.unrealengine.com/course/2504330",
                    "confidence": 0.9
                },
                {
                    "name": "Unreal C++ Tutorial by Virtus Learning Hub",
                    "type": "YouTube Tutorial",
                    "url": "https://youtube.com/playlist?list=PL4mRaHZim4URe6RxpGadr_7Wl1WRY4uqI&si=Yd2QZfWA0Bl7iApD",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Unreal Engine Documentation",
                    "url": "https://docs.unrealengine.com/",
                    "confidence": 0.95
                },
                {
                    "name": "Unreal C++ API Reference",
                    "url": "https://docs.unrealengine.com/en-US/API/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Unreal Engine", "Game Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    
    @Rule(
        Answer(ident="skill", text="Game Development"),
        Answer(ident="sub_skill", text="Unreal Engine"),
        Answer(ident="level", text="Advanced"),
        salience=1000
    )
    def recommend_unreal_advanced(self):
        details = {
            "ide": {
                "name": "Visual Studio",
                "url": "https://visualstudio.microsoft.com/",
                "description": "Professional IDE for Unreal C++ development with advanced debugging.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Unreal Advanced Programming",
                    "type": "Interactive Tutorial",
                    "url": "https://learn.unrealengine.com/course/2504331",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Unreal Engine Documentation",
                    "url": "https://docs.unrealengine.com/",
                    "confidence": 0.95
                },
                {
                    "name": "Unreal Chaos Physics Documentation",
                    "url": "https://docs.unrealengine.com/en-US/Engine/Physics/ChaosPhysics/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Unreal Engine", "Game Development")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    # --------- Data Science - Data Analysis ---------
    
    @Rule(
        Answer(ident="skill", text="Data Science"),
        Answer(ident="sub_skill", text="Data Analysis"),
        Answer(ident="level", text="Beginner"),
        salience=1000
    )
    def recommend_data_analysis_beginner(self):
        details = {
            "ide": {
                "name": "Jupyter Notebook",
                "url": "https://jupyter.org/",
                "description": "Interactive IDE for Python-based data analysis, ideal for beginners.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Python for Data Analysis by Kaggle",
                    "type": "Interactive Tutorial",
                    "url": "https://www.kaggle.com/learn/python",
                    "confidence": 0.9
                },
                {
                    "name": "Data Analysis with Pandas by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://youtu.be/gtjxAH8uaP0?si=IKfaPrSEFmrOwhxs",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Pandas Documentation",
                    "url": "https://pandas.pydata.org/docs/",
                    "confidence": 0.95
                },
                {
                    "name": "NumPy Documentation",
                    "url": "https://numpy.org/doc/stable/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Data Analysis", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(
        Answer(ident="skill", text="Data Science"),
        Answer(ident="sub_skill", text="Data Analysis"),
        Answer(ident="level", text="Intermediate"),
        salience=1000
    )
    def recommend_data_analysis_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Versatile IDE with Python extensions for data analysis workflows.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Intermediate Data Analysis by DataCamp",
                    "type": "Interactive Tutorial",
                    "url": "https://www.datacamp.com/tracks/data-analyst-with-python",
                    "confidence": 0.9
                },
                {
                    "name": "Pandas Tutorial by Corey Schafer",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/playlist?list=PL-osiE80TeTsWmV9i9c58mdDCSskIFdDS",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Pandas Documentation",
                    "url": "https://pandas.pydata.org/docs/",
                    "confidence": 0.95
                },
                {
                    "name": "SciPy Documentation",
                    "url": "https://docs.scipy.org/doc/scipy/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Data Analysis", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    @Rule(
        Answer(ident="skill", text="Data Science"),
        Answer(ident="sub_skill", text="Data Analysis"),
        Answer(ident="level", text="Advanced"),
        salience=1000
    )
    def recommend_data_analysis_advanced(self):
        details = {
            "ide": {
                "name": "PyCharm Professional",
                "url": "https://www.jetbrains.com/pycharm/",
                "description": "Advanced IDE with Python support for large-scale data analysis.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Advanced Data Analysis with Python",
                    "type": "Video Course",
                    "url": "https://www.udemy.com/course/data-analysis-with-pandas/",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "Pandas Documentation",
                    "url": "https://pandas.pydata.org/docs/",
                    "confidence": 0.95
                },
                {
                    "name": "Dask Documentation",
                    "url": "https://docs.dask.org/en/stable/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Data Analysis", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    # --------- Data Science - Machine Learning ---------
    
    @Rule(
        Answer(ident="skill", text="Data Science"),
        Answer(ident="sub_skill", text="Machine Learning"),
        Answer(ident="level", text="Beginner"),
        salience=1000
    )
    def recommend_ml_beginner(self):
        details = {
            "ide": {
                "name": "Google Colab",
                "url": "https://colab.research.google.com/",
                "description": "Cloud-based Jupyter environment for ML, beginner-friendly with free GPU.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Intro to Machine Learning by Kaggle",
                    "type": "Interactive Tutorial",
                    "url": "https://www.kaggle.com/learn/intro-to-machine-learning",
                    "confidence": 0.9
                },
                {
                    "name": "Machine Learning Crash Course by Google",
                    "type": "Video Course",
                    "url": "https://developers.google.com/machine-learning/crash-course",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "scikit-learn Documentation",
                    "url": "https://scikit-learn.org/stable/documentation.html",
                    "confidence": 0.95
                },
                {
                    "name": "Machine Learning Cheatsheet",
                    "url": "https://www.codecademy.com/learn/machine-learning/modules/supervised-machine-learning/cheatsheet",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Machine Learning", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    
    @Rule(
        Answer(ident="skill", text="Data Science"),
        Answer(ident="sub_skill", text="Machine Learning"),
        Answer(ident="level", text="Intermediate"),
        salience=1000
    )
    def recommend_ml_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Flexible IDE with Python and ML libraries for model development.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Intermediate Machine Learning by Kaggle",
                    "type": "Interactive Tutorial",
                    "url": "https://www.kaggle.com/learn/intermediate-machine-learning",
                    "confidence": 0.9
                },
                {
                    "name": "ML with scikit-learn by Data School",
                    "type": "YouTube Tutorial",
                    "url": "https://www.youtube.com/playlist?list=PL5-da3qGB5ICeMbQuqbbCOQWcS6OYBr5A",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "scikit-learn Documentation",
                    "url": "https://scikit-learn.org/stable/documentation.html",
                    "confidence": 0.95
                },
                {
                    "name": "XGBoost Documentation",
                    "url": "https://xgboost.readthedocs.io/en/stable/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Machine Learning", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
    
    @Rule(
        Answer(ident="skill", text="Data Science"),
        Answer(ident="sub_skill", text="Machine Learning"),
        Answer(ident="level", text="Advanced"),
        salience=1000
    )
    def recommend_ml_advanced(self):
        details = {
            "ide": {
                "name": "PyCharm Professional",
                "url": "https://www.jetbrains.com/pycharm/",
                "description": "Advanced IDE for ML model deployment and optimization.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Machine Learning by Andrew Ng",
                    "type": "Video Course",
                    "url": "https://www.coursera.org/learn/machine-learning",
                    "confidence": 0.9
                },
                {
                    "name": "Advanced ML Techniques",
                    "type": "Article Series",
                    "url": "https://towardsdatascience.com/tagged/machine-learning",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "scikit-learn Documentation",
                    "url": "https://scikit-learn.org/stable/documentation.html",
                    "confidence": 0.95
                },
                {
                    "name": "MLflow Documentation",
                    "url": "https://mlflow.org/docs/latest/index.html",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Machine Learning", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Data Science"),
      Answer(ident="sub_skill", text="Deep Learning"),
      Answer(ident="level", text="Beginner"),
      salience=1000)
    def recommend_deep_learning_beginner(self):
        details = {
            "ide": {
                "name": "Google Colab",
                "url": "https://colab.research.google.com/",
                "description": "Cloud-based IDE with GPU support, perfect for deep learning beginners.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Intro to Deep Learning by Kaggle",
                    "type": "Interactive Tutorial",
                    "url": "https://www.kaggle.com/learn/intro-to-deep-learning",
                    "confidence": 0.9
                },
                {
                    "name": "Deep Learning Crash Course by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://youtu.be/VyWAvY2CF9c?si=Niy7Rm34Z4ZL-6Uz",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "TensorFlow Documentation",
                    "url": "https://www.tensorflow.org/learn",
                    "confidence": 0.95
                },
                {
                    "name": "Keras Documentation",
                    "url": "https://keras.io/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Deep Learning", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Data Science"),
      Answer(ident="sub_skill", text="Deep Learning"),
      Answer(ident="level", text="Intermediate"),
      salience=1000)
    def recommend_deep_learning_intermediate(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Versatile IDE with TensorFlow and PyTorch for deep learning models.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Deep Learning Specialization by Coursera",
                    "type": "Video Course",
                    "url": "https://www.coursera.org/specializations/deep-learning",
                    "confidence": 0.9
                },
                {
                    "name": "PyTorch Tutorial by Sentdex",
                    "type": "YouTube Tutorial",
                    "url": "https://youtube.com/playlist?list=PLQVvvaa0QuDdeMyHEYc0gxFpYwHY2Qfdh&si=Y1LlAy-jGtdEKwP0",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "TensorFlow Documentation",
                    "url": "https://www.tensorflow.org/learn",
                    "confidence": 0.95
                },
                {
                    "name": "PyTorch Documentation",
                    "url": "https://pytorch.org/docs/stable/index.html",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Deep Learning", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Data Science"),
      Answer(ident="sub_skill", text="Deep Learning"),
      Answer(ident="level", text="Advanced"),
      salience=1000)
    def recommend_deep_learning_advanced(self):
        details = {
            "ide": {
                "name": "PyCharm Professional",
                "url": "https://www.jetbrains.com/pycharm/",
                "description": "Advanced IDE for deep learning model training and deployment.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Deep Learning by fast.ai",
                    "type": "Video Course",
                    "url": "https://course.fast.ai/",
                    "confidence": 0.9
                },
                {
                    "name": "Advanced Deep Learning",
                    "type": "Article Series",
                    "url": "https://www.deeplearning.ai/resources/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "TensorFlow Documentation",
                    "url": "https://www.tensorflow.org/learn",
                    "confidence": 0.95
                },
                {
                    "name": "Hugging Face Transformers",
                    "url": "https://huggingface.co/docs/transformers/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Deep Learning", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
    
        
    @Rule(Answer(ident="skill", text="Data Science"),
      Answer(ident="sub_skill", text="Data Visualization"),
      Answer(ident="level", text="Beginner"),
      salience=1000)
    def recommend_data_visualization_beginner(self):
        details = {
            "ide": {
                "name": "Jupyter Notebook",
                "url": "https://jupyter.org/",
                "description": "Interactive IDE for creating visualizations with Python, ideal for beginners.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Data Visualization with Python by Kaggle",
                    "type": "Interactive Tutorial",
                    "url": "https://www.kaggle.com/learn/data-visualization",
                    "confidence": 0.9
                },
                {
                    "name": "Matplotlib Tutorial by freeCodeCamp",
                    "type": "YouTube Tutorial",
                    "url": "https://youtu.be/3Xc3CA655Y4?si=RzryfFPVMZJcysWE",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "Matplotlib Documentation",
                    "url": "https://matplotlib.org/stable/contents.html",
                    "confidence": 0.95
                },
                {
                    "name": "Seaborn Documentation",
                    "url": "https://seaborn.pydata.org/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.9
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Beginner", "Data Visualization", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))
      
        
    @Rule(Answer(ident="skill", text="Data Science"),
      Answer(ident="sub_skill", text="Data Visualization"),
      Answer(ident="level", text="Intermediate"),
      salience=1000)
    def recommend_data_visualization_intermediate(self):
        details = {
            "ide": {
                "name": "RStudio",
                "url": "https://www.rstudio.com/",
                "description": "IDE for R-based visualizations, great for statistical plotting.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "Tableau Tutorial by Tableau",
                    "type": "Video Course",
                    "url": "https://www.tableau.com/learn/training",
                    "confidence": 0.9
                }
            ],
            "documentation": [
                {
                    "name": "ggplot2 Documentation",
                    "url": "https://ggplot2.tidyverse.org/",
                    "confidence": 0.95
                },
                {
                    "name": "Tableau Help",
                    "url": "https://help.tableau.com/current/pro/desktop/en-us/help.htm",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.85
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Intermediate", "Data Visualization", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="skill", text="Data Science"),
      Answer(ident="sub_skill", text="Data Visualization"),
      Answer(ident="level", text="Advanced"),
      salience=1000)
    def recommend_data_visualization_advanced(self):
        details = {
            "ide": {
                "name": "VS Code",
                "url": "https://code.visualstudio.com/",
                "description": "Advanced IDE for interactive visualizations with JavaScript and Python.",
                "confidence": 0.95
            },
            "learning_resources": [
                {
                    "name": "D3.js Tutorial",
                    "type": "Interactive Tutorial",
                    "url": "https://d3js.org/getting-started",
                    "confidence": 0.9
                },
                {
                    "name": "Advanced Visualization with Plotly",
                    "type": "Video Course",
                    "url": "https://www.udemy.com/course/interactive-python-dashboards-with-plotly-and-dash/",
                    "confidence": 0.85
                }
            ],
            "documentation": [
                {
                    "name": "D3.js Documentation",
                    "url": "https://d3js.org/",
                    "confidence": 0.95
                },
                {
                    "name": "Plotly Documentation",
                    "url": "https://plotly.com/python/",
                    "confidence": 0.9
                }
            ]
        }
        confidence = 0.8
        confidence = self.combine_confidence(confidence, details["ide"]["confidence"])
        for resource in details["learning_resources"]:
            confidence = self.combine_confidence(confidence, resource["confidence"])
        for doc in details["documentation"]:
            confidence = self.combine_confidence(confidence, doc["confidence"])
        confidence = max(-1.0, min(1.0, confidence))
        self.display_recommendation(details, confidence, "Advanced", "Data Visualization", "Data Science")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

        #----- End of Recommendation Rules 📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌📌------
    
    @Rule(Answer(ident="skill", text=MATCH.skill),
          Answer(ident="sub_skill", text=MATCH.sub_skill),
          Answer(ident="level", text=MATCH.level),
          NOT(Fact(ask="continue_choice")),
          TEST(lambda skill, sub_skill, level: not (
              skill == "Programming" and sub_skill in ["Python", "Java", "C++"] and level in ["Beginner", "Intermediate", "Advanced"]
          )),
          salience=900)
    def no_recommendation(self, skill, sub_skill, level):
        self.recommend_action(f"⚠️ No recommendations found for {skill} ({sub_skill}, {level}). Please try again with a different selection.")
        self.recommendation_found = True
        self.declare(Fact(ask="continue_choice"))

    @Rule(Answer(ident="continue_choice", text="yes"),
          salience=1000)
    def handle_continue_yes(self):
        self.recommendation_found = False
        facts_to_retract = []
        for fact_id, fact in self.facts.items():
            if isinstance(fact, Answer) or 'ask' in fact:
                facts_to_retract.append(fact)
        for fact in facts_to_retract:
            self.retract(fact)
        try:
            start_fact = [f for f in self.facts.values() if 'start' in f]
            if start_fact:
                self.retract(start_fact[0])
        except IndexError:
            pass             
        self.declare(Fact(start=True))
        self.declare(Fact(ask="skill"))

    @Rule(Answer(ident="continue_choice", text="no"),
          salience=1000)
    def handle_continue_no(self):
        self.recommend_action("🤗 Recommendation session complete. We appreciate your time!")
        self.halt()


if __name__ == "__main__":
    engine = LearningAdvisor()
    engine.reset()
    engine.run()
    

🔎 The expert system is ready to help you.
Let's get started with your recommendations!


🎓 Choose the field you want to learn:
1. Game Development
2. Data Science
3. Mobile App Development
4. Web Development
5. Programming
➤ Please enter the option number:


 5



🎯 Awesome! What would you like to focus on in Programming?
1. Python
2. Java
3. C++
4. JavaScript
5. C#
6. Ruby
➤ Please enter the option number:


 1



📈 What is your current skill level?
1. Beginner
2. Intermediate
3. Advanced
➤ Please enter the option number:


...


📈 What is your current skill level?
1. Beginner
2. Intermediate
3. Advanced
➤ Please enter the option number:


 2



✨ Recommendation for learning Data Science (Deep Learning, Intermediate):
Overall Confidence: 0.9999994375
------------------------------------------------------------
🛠️ IDE:
  Name       : VS Code
  Description: Versatile IDE with TensorFlow and PyTorch for deep learning models.
  URL        : https://code.visualstudio.com/
------------------------------------------------------------
📚 Learning Resources:
  - Deep Learning Specialization by Coursera (Video Course): https://www.coursera.org/specializations/deep-learning
  - PyTorch Tutorial by Sentdex (YouTube Tutorial): https://youtube.com/playlist?list=PLQVvvaa0QuDdeMyHEYc0gxFpYwHY2Qfdh&si=Y1LlAy-jGtdEKwP0
------------------------------------------------------------
📄 Documentation:
  - TensorFlow Documentation: https://www.tensorflow.org/learn
  - PyTorch Documentation: https://pytorch.org/docs/stable/index.html


🔄 Would you like to get another recommendation? (yes/no):
1. yes
2. no
➤ Please enter the option number:


 no


🤗 Recommendation session complete. We appreciate your time!

